In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime

# Read data

In [2]:
def get_current_data():
    current_date = date.today().strftime("%Y%m%d")
    data_file = "icon-eu-eps_"+current_date+"00_{}_Karlsruhe.txt"
    #Read data
    try:
        temp_data = pd.read_csv("data/"+data_file.format("t_2m"), sep = "|", skiprows = 3).dropna(axis = 1)
        temp_data.columns = temp_data.columns.str.replace(" ", "")
        wind_data = pd.read_csv("data/"+data_file.format("wind_mean_10m"), sep = "|", skiprows = 3).dropna(axis = 1)
        wind_data.columns = wind_data.columns.str.replace(" ", "")
    except:
        print("Data not available")
    return temp_data, wind_data

In [3]:
temp_data, wind_data = get_current_data()

## Create basic quantile prediction

In [4]:
horizons = [36, 48, 60, 72, 84]
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

In [5]:
def get_naive_prediction(data, quantiles, horizons):
    prediction = pd.DataFrame(index = quantiles, columns = horizons)
    for horizon in horizons:
        data_horizon = data[data["fcst_hour"] == horizon].drop("fcst_hour", axis = 1)
        pred_quantiles = np.quantile(data_horizon, quantiles)
        prediction.loc[:,horizon] = pred_quantiles
    return prediction

In [6]:
temp_pred = get_naive_prediction(temp_data, quantiles, horizons)
wind_pred = get_naive_prediction(wind_data, quantiles, horizons)

In [10]:
# Prepare final format
def get_final_format(data, target, save = True):
    final_prediction = pd.DataFrame(columns = ["forecast_date","target","horizon","q0.025","q0.25","q0.5","q0.75","q0.975"], index = np.arange(0,5))
    final_prediction["forecast_date"] = datetime.today().strftime("%Y-%m-%d")
    final_prediction["horizon"] = ["{} hour".format(x) for x in horizons]
    final_prediction["target"] = target

    for quantile in quantiles:
        final_prediction.loc[:,"q{}".format(quantile)] = data.loc[quantile].values
    #Save data
    if save == True:        
        final_prediction.to_pickle("../evaluation/predictions/single/{}_{}".format(target, date.today().strftime("%Y-%m-%d")))
    else:
        return final_prediction   

In [11]:
get_final_format(temp_pred, "temperature")
get_final_format(wind_pred, "wind")